## Recruitment Process Workflow (First Time)

This code is used to process data dowloaded from iCIMS and prepare it for Power BI reporting.

**Step 1:** We import  the data that we downloaded from iCIMS 

In [4]:
import pandas as pd
import numpy as np 

# Load the data
data = pd.read_csv('Recruitment_Data.csv', encoding='ISO-8859-1')

**Step 2:** We convert date/times to dates only; then create a new column for *Application Year*

In [5]:
print(data['Application Date'].dtype)

object


In [6]:
# Identify columns that have dates and times and convert to just dates
for column in data.columns:
    if column != 'StatusID': # Skip the StatusID column (it is an integer)
        try:
            # Attempt to convert the column to datetime
            temp = pd.to_datetime(data[column])
            # If successful, and no errors are raised, convert to just date
            data[column] = temp.dt.date
        except ValueError:
            # If error occurs, it means this column cannot be converted to datetime
            continue
            
# Extract the year from 'Application Date' and create a new column 'Application Year'
data['Application Year'] = data['Application Date'].apply(lambda x: x.year if pd.notnull(x) else None)

**Step 3:** We use StatusIDs to identify "Closed" applications.

In [7]:
## Here is a mapping of the Status with Status IDs
#################################################
# Decision Pending: 2024, 8769
# Hired: 17582
# In Review: 2020
# Initial DNQ: 2017
# Interview Completed: 8768
# Interview Requested: 25794
# Interview Scheduled: 8767
# Interviewed; Not Selected: 2038
# Offer Accepted: 2036
# Offer Declined/Rejected: 2035
# Offer Requested: 2031
# Offer Rescinded: 8772
# Phone Screen Completed: 8766
# Phone Screen Completed; Not Selected: 2026
# Phone Screen Scheduled: 8765
# Reviewed; Not Selected: 15006, 15007
# Self-Withdrew (Portal): 8440
# Self-Withdrew (Recruiter): 20186
# Submitted to Hiring Manager: 2028

# Define the list of StatusIDs that indicate a Closed record
closed_status_ids = {17582, 2017, 2038, 2035, 8772, 2026, 15006, 15007, 8440, 20186}


# Create a new column 'Record Status' based on whether StatusID is in the closed list, place it after 'StatusID'
data.insert(loc=data.columns.get_loc('StatusID') + 1, column='Record Status', 
            value=data['StatusID'].apply(lambda x: 'Closed' if x in closed_status_ids else 'Active'))

**Step 4:** We create a new column called *Stage*, which converts the *Bin* text to a numbered stage of 2 through 5.\
Note: Stage 1 is initial Application.

In [8]:
# Define conditions for the 'Stage' column based on 'Bin'
conditions = [
    (data['Bin'] == 'New Submissions'),
    (data['Bin'] == 'HR/ Recruiter Review'),
    (data['Bin'] == 'Hiring Manager Review'),
    (data['Bin'] == 'Offer'),
    (data['Bin'] == 'Hired')
]
# Define the corresponding stage values
choices = [1, 2, 3, 4, 5]

# Create the 'Stage' column with default value 99
data['Stage'] = 99

# Apply conditions and choices to the 'Stage' column
data.loc[:, 'Stage'] = np.select(conditions, choices, default=99)

# Insert the 'Stage' column right after the 'Bin' column
data.insert(loc=data.columns.get_loc('Bin') + 1, column='Stage', value=data.pop('Stage'))

**Step 5:** We create a new column called *Last Action*. Look through all closed applications, finds the latest date, and identify the name of that column.
Then map those column name to the appropriate stage of the process.\
What phase was the application in when it was closed? 

In [9]:
# Define a function to find the last action based on the dates in each row, ignoring 'First Candidate Withdrew' for the latest date
def find_last_action(row):
    if row['Record Status'] == 'Active':
        return ''  # Leave blank for Active records
    elif row['Record Status'] == 'Closed':
        # Get the latest date and the corresponding column name, excluding 'StatusID' and 'First Candidate Withdrew'
        latest_date = None
        latest_column = 'Application'  # Default action if no other dates are available
        for col in data.columns:
            if data[col].dtype == 'object' and col != 'StatusID' and col != 'First Candidate Withdrew':
                try:
                    date = pd.to_datetime(row[col])
                    if not pd.isnull(date) and (latest_date is None or date > latest_date):
                        latest_date = date
                        latest_column = col
                except:
                    continue
        return latest_column
    
#Define a funtion to map column names to new descriptions/bins
def map_column_names(last_action):
   # Define the mapping dictionary
    mapping_dict = {
        'Application Date': 'Application',
        'First Hired: Hired': 'Hired',
        'First Hiring Manager Review: Interview Requested': 'Hiring Manager Review',
        'First Hiring Manager Review: Interviewed; Not Selected': 'Hiring Manager Review',
        'First Hiring Manager Review: Submitted to Hiring Manager': 'Hiring Manager Review',
        'First HR/ Recruiter Review': 'HR/Recruiter Review',
        'First HR/ Recruiter Review: In Review': 'HR/Recruiter Review',
        'First HR/ Recruiter Review: Reviewed; Not Selected': 'HR/Recruiter Review',
        'First Offer: Offer Accepted': 'Offer',
        'First Offer: Offer Declined/Rejected': 'Offer',
        'First Offer: Offer Extended': 'Offer',
        'First Offer: Offer Rescinded': 'Offer'
    }
    # Return the mapped value if it exists, otherwise return the original value
    return mapping_dict.get(last_action, last_action)

# Apply the function to each row to determine the 'Last Action'
data['Last Action'] = data.apply(find_last_action, axis=1)

# Apply the function to map the 'Last Action' names to new values
data['Last Action'] = data['Last Action'].apply(map_column_names)

**Step 6:** Create a new column *StartOfActivePhase* that records the start date of the latest stage for "Active" applications


In [10]:
# Define a function to find the start date of the latest active phase
def find_start_of_active_phase(row):
    if row['Record Status'] == 'Active':
        latest_date = None
        for col in data.columns:
            if col != 'StatusID' and data[col].dtype == 'object':  # Skip StatusID and non-date columns
                try:
                    date = pd.to_datetime(row[col], errors='coerce')
                    if not pd.isnull(date) and (latest_date is None or date > latest_date):
                        latest_date = date
                except:
                    continue
        return latest_date
    return None

# Apply the function to determine the 'StartOfActivePhase'
data['StartOfActivePhase'] = data.apply(find_start_of_active_phase, axis=1)

# Convert 'StartOfActivePhase' to dt.date format
data['StartOfActivePhase'] = data['StartOfActivePhase'].apply(lambda x: x.date() if pd.notnull(x) else None)

**Step 7:** Create columns 'Start_HR/Recruiter', 'End_HR/Recruiter', 'Start_HiringManager', 'End_HiringManager', 'Start_Offer', and 'End_Offer' for closed applications

In [11]:
# Define a function to find the start and end dates for HR/Recruiter, Hiring Manager, and Offer stages, 
# and the end date for the Offer stage

def find_stage_dates(row):
    if row['Record Status'] == 'Closed':
        # Initialize variables for the dates
        start_hr_recruiter = None
        end_hr_recruiter = None
        start_hiring_manager = None
        end_hiring_manager = None
        start_offer = None
        end_offer = None
        
        # Find the earliest date for the HR/Recruiter stage
        hr_recruiter_columns = [
            'First HR/ Recruiter Review',
            'First HR/ Recruiter Review: In Review',
            'First HR/ Recruiter Review: Reviewed; Not Selected'
        ]
        for col in hr_recruiter_columns:
            try:
                date = pd.to_datetime(row[col], errors='coerce')
                if not pd.isnull(date) and (start_hr_recruiter is None or date < start_hr_recruiter):
                    start_hr_recruiter = date
            except:
                continue
        
        # Find the earliest date for the Hiring Manager stage
        hiring_manager_columns = [
            'First Hiring Manager Review: Submitted to Hiring Manager',
            'First Hiring Manager Review: Interview Requested',
            'First Hiring Manager Review: Interviewed; Not Selected'
        ]
        for col in hiring_manager_columns:
            try:
                date = pd.to_datetime(row[col], errors='coerce')
                if not pd.isnull(date) and (start_hiring_manager is None or date < start_hiring_manager):
                    start_hiring_manager = date
            except:
                continue
        
        # Set End_HR/Recruiter based on Start_HiringManager or latest date from specified columns
        if pd.notnull(start_hr_recruiter):
            if pd.notnull(start_hiring_manager):
                end_hr_recruiter = start_hiring_manager
            else:
                end_hr_recruiter_columns = [
                    'First HR/ Recruiter Review: Reviewed; Not Selected',
                    'First Hired: Hired',
                    'First Candidate Withdrew'
                ]
                for col in end_hr_recruiter_columns:
                    try:
                        date = pd.to_datetime(row[col], errors='coerce')
                        if not pd.isnull(date) and (end_hr_recruiter is None or date > end_hr_recruiter):
                            end_hr_recruiter = date
                    except:
                        continue
        
        # Find the date for the Offer stage
        try:
            start_offer = pd.to_datetime(row['First Offer: Offer Extended'], errors='coerce')
        except:
            start_offer = None
        
        # Set End_HiringManager based on Start_Offer or latest date from specified columns
        if pd.notnull(start_hiring_manager):
            if pd.notnull(start_offer):
                end_hiring_manager = start_offer
            else:
                end_hiring_manager_columns = [
                    'First Hiring Manager Review: Interviewed; Not Selected',
                    'First Hired: Hired',
                    'First Candidate Withdrew'
                ]
                for col in end_hiring_manager_columns:
                    try:
                        date = pd.to_datetime(row[col], errors='coerce')
                        if not pd.isnull(date) and (end_hiring_manager is None or date > end_hiring_manager):
                            end_hiring_manager = date
                    except:
                        continue
        
        # Find the latest date for the end of the Offer stage, only if 'Start_Offer' is not blank
        if pd.notnull(start_offer):
            end_offer_columns = [
                'First Offer: Offer Accepted',
                'First Offer: Offer Declined/Rejected',
                'First Offer: Offer Rescinded',
                'First Hired: Hired',
                'First Candidate Withdrew'
            ]
            for col in end_offer_columns:
                try:
                    date = pd.to_datetime(row[col], errors='coerce')
                    if not pd.isnull(date) and (end_offer is None or date > end_offer):
                        end_offer = date
                except:
                    continue
        
        return start_hr_recruiter, end_hr_recruiter, start_hiring_manager, end_hiring_manager, start_offer, end_offer
    return None, None, None, None, None, None

# Apply the function to determine the stage dates
data[['Start_HR/Recruiter', 'End_HR/Recruiter', 'Start_HiringManager', 'End_HiringManager', 'Start_Offer', 'End_Offer']] = data.apply(
    lambda row: pd.Series(find_stage_dates(row)), axis=1
)

# Convert the stage dates to dt.date format
data['Start_HR/Recruiter'] = data['Start_HR/Recruiter'].apply(lambda x: x.date() if pd.notnull(x) else None)
data['End_HR/Recruiter'] = data['End_HR/Recruiter'].apply(lambda x: x.date() if pd.notnull(x) else None)
data['Start_HiringManager'] = data['Start_HiringManager'].apply(lambda x: x.date() if pd.notnull(x) else None)
data['End_HiringManager'] = data['End_HiringManager'].apply(lambda x: x.date() if pd.notnull(x) else None)
data['Start_Offer'] = data['Start_Offer'].apply(lambda x: x.date() if pd.notnull(x) else None)
data['End_Offer'] = data['End_Offer'].apply(lambda x: x.date() if pd.notnull(x) else None)

**Step 8:** Expert the processed data to a new CSV file.

In [12]:
# Save the processed data to a new CSV file
data.to_csv('Processed_Recruiting_Data.csv', index=False)

In [13]:
# Separate Active and Closed applications
active_data = data[data['Record Status'] == 'Active']
closed_data = data[data['Record Status'] == 'Closed']

In [14]:
# Save the Active applications
active_data.to_csv('Processed_Active_Applications.csv', index=False)
# Save the Closed applications
closed_data.to_csv('Processed_Closed_Applications.csv', index=False)